# Laboratorio 0 — Preparación del entorno y buenas prácticas

Este cuaderno define convenciones (reproducibilidad, estructura de experimentos y métricas) que se reutilizarán en el resto de prácticas.

## Objetivos
- Verificar el entorno (versiones, GPU si aplica).
- Establecer semillas y una estructura mínima de experimento.
- Adoptar un flujo de trabajo repetible: datos → modelo → evaluación → registro.

## Requisitos
- Python 3.10+ recomendado
- Jupyter
- numpy, pandas, matplotlib, scikit-learn


## 1) Comprobación del entorno

Ejecuta la siguiente celda para ver versiones. Si alguna librería falta, instala con `pip install ...` (idealmente dentro de un entorno virtual).


In [1]:
import sys, platform
import numpy as np
import pandas as pd
import sklearn
import matplotlib

print("Python:", sys.version.split()[0])
print("Plataforma:", platform.platform())
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Scikit-learn:", sklearn.__version__)
print("Matplotlib:", matplotlib.__version__)


Python: 3.14.0
Plataforma: Windows-11-10.0.26100-SP0
NumPy: 2.3.4
Pandas: 2.3.3
Scikit-learn: 1.8.0
Matplotlib: 3.10.7


## 2) Reproducibilidad

En ciencia de datos es habitual obtener resultados distintos entre ejecuciones por aleatoriedad (barajado de datos, inicialización de pesos, etc.). 
Aquí fijamos semillas y definimos utilidades para evaluar de forma consistente.


In [2]:
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print("Semilla fijada:", SEED)


Semilla fijada: 42


## 3) Utilidades de evaluación

Definimos dos funciones prácticas: una para dividir conjuntos y otra para reportar métricas de forma estándar.
Más adelante se extenderán para CV y NLP.


In [ ]:
from dataclasses import dataclass
from typing import Dict, Any, Tuple

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score, accuracy_score, f1_score

"""
Dividir los datos en Datos de entrenamiento y Datos test
Usa siempre la misma semilla (para tener siempre los mismos resultados)
"""
def split(X, y, test_size=0.2, seed=42):
    return train_test_split(X, y, test_size=test_size, random_state=seed)

"""
Función que recibe valores reales (y_true) y valores predichos (y_pred)
"""
def regression_report(y_true, y_pred) -> Dict[str, float]:
    # Lo devuelve como float para poder guardarlo en JSON.
    return {
        # MAE: error promedio
        "MAE": float(mean_absolute_error(y_true, y_pred)),

        # Diferencia promedio entre los valores que predice tu modelo y los valores reales
        # RMSE: error penalizando errores grandes
        "RMSE": float(root_mean_squared_error(y_true, y_pred, squared=False)),
        
        # Proporción de la variabilidad de la variable dependiente es explicada por el modelo
        # R²: qué tan bien explica el modelo los datos
        "R2": float(r2_score(y_true, y_pred)),
    }


"""
Función para modelos de clasificación.

Accuracy: porcentaje de aciertos
F1 macro: balance entre precisión y recall
"""
def classification_report_simple(y_true, y_pred) -> Dict[str, float]:
    return {
        "Accuracy": float(accuracy_score(y_true, y_pred)),
        "F1 (macro)": float(f1_score(y_true, y_pred, average="macro")),
    }


## 4) Registro mínimo de experimentos

No hace falta una plataforma compleja para empezar; basta con guardar:
- configuración (modelo/hiperparámetros),
- métricas,
- fecha/hora y semilla.

Esto permite comparar versiones sin perderse.


In [ ]:
import json
from datetime import datetime
from pathlib import Path

"""
name: nombre del experimento
config: cómo entrenaste el modelo
metrics: resultados
X: dataset (opcional)
out_dir: carpeta donde guardar
"""
def log_experiment(name: str, config: Dict[str, Any], metrics: Dict[str, Any], out_dir="experimentos"):
    # Crea la carpeta si no existe.
    Path(out_dir).mkdir(exist_ok=True)

    # Toda la información del experimento junta.
    payload = {
        "name": name,
        "timestamp": datetime.now().isoformat(timespec="seconds"),
        "seed": SEED,
        "config": config,
        "metrics": metrics,
    }

    # Crear una archivo json con todos los datos, fecha y hora - Guarda el archivo en .json
    path = Path(out_dir) / f"{name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    path.write_text(json.dumps(payload, ensure_ascii=False, indent=2))
    return str(path)

print("Listo. Usa log_experiment(...) en los laboratorios.")


Listo. Usa log_experiment(...) en los laboratorios.


## Ejercicios

### Ejercicio 1: Entorno reproducible
Crea un entorno virtual (venv/conda) e instala las librerías necesarias. Exporta un `requirements.txt` mínimo para estos laboratorios.

**Entregables**
- Comando(s) usados
- Archivo `requirements.txt`

**Criterios de evaluación**
- El entorno se crea y activa correctamente
- El requirements incluye versiones o, como mínimo, librerías clave
- Se documenta cómo reproducirlo


-------------------------------------------------------------
### Comandos para desplegar entorno
**Reproducir entorno virtual en Windows 11**

1. PowerShell -> Crear entorno -- Dentro de la carpeta de nuestro codigo - crea una carpeta env

    python -m venv env

2. Ejecutar en PowerShell:

    Set-ExecutionPolicy -ExecutionPolicy RemoteSigned -Scope Process

3. Activar entorno
    
    .\env\Scripts\Activate.ps1

4. Instalar librerías

    pip install pandas matplotlib scikit-learn joblib

5. Exportar requirements
    
    pip freeze > requirements.txt

3. Desactivar entorno
    
    deactivate
-------------------------------------------------------------


### Ejercicio 2: Plantilla de experimento
Extiende `log_experiment` para que también guarde el tamaño del dataset y el nombre de las columnas/variables.

**Entregables**
- Función modificada
- Ejemplo de uso con un dataset pequeño

**Pistas**
- `X.shape` y `list(X.columns)` si trabajas con pandas

**Criterios de evaluación**
- Registra información adicional útil
- No rompe compatibilidad con el uso anterior
- El JSON queda legible

In [ ]:
"""
Definir en la funcion que queremos modificar un nuevo parámetro (X) darle el valor de NONE por si no tiene valor.
Agregar en el nuevo parámetro al playload (El conjunto de datos que queremos guardar, enviar o registrar como una sola unidad)
"""
def log_experiment(name: str, config: Dict[str, Any], metrics: Dict[str, Any], X: pd.DataFrame | None = None, out_dir="experimentos"):
    dataset_info = None

    if X is not None:
        dataset_info = {
            "n_rows": X.shape[0],
            "n_columns": X.shape[1],
            "columns": list(X.columns)
        }
    
    Path(out_dir).mkdir(exist_ok=True)
    payload = {
        "name": name,
        "timestamp": datetime.now().isoformat(timespec="seconds"),
        "seed": SEED,
        "config": config,
        "metrics": metrics,
        "dataset": dataset_info
    }
    path = Path(out_dir) / f"{name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    path.write_text(json.dumps(payload, ensure_ascii=False, indent=2))
    return str(path)


# Dataset de ejemplo
df = pd.DataFrame({
    "edad": [25, 32, 40, 28],
    "ingresos": [30000, 45000, 60000, 38000]
})

# Simulamos métricas
metrics = {
    "MAE": 1234.5,
    "RMSE": 1500.2,
    "R2": 0.82
}

# Configuración ficticia
config = {
    "model": "LinearRegression",
    "test_size": 0.2
}

log_experiment(
    name="regresion_demo",
    config=config,
    metrics=metrics,
    X=df
)

print("")

'experimentos\\regresion_demo_20260122_171051.json'